In [163]:
from langchain.agents import load_tools,initialize_agent,AgentType,Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
import os
from dotenv import load_dotenv

In [143]:
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")
os.environ["SERPER_API_KEY"]=os.getenv("SERPER_API_KEY")
api_key=os.getenv("GOOGLE_API_KEY")

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]

In [24]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [164]:
llm=ChatGoogleGenerativeAI(model='gemini-pro',convert_system_message_to_human=True)
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]

In [170]:

agent=initialize_agent(tools=tools,llm=llm,agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [172]:
agent.run("what is my current directory")



> Entering new AgentExecutor chain...


c:\LLMs\MultipleToolKit\toolkit\Lib\site-packages\langchain_google_genai\chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Question: what is my current directory
Thought: I need to find my current directory
Action:
```
{
  "action": "Intermediate Answer",
  "action_input": {
    "question": "How do I find my current directory in a terminal?"
  }
}
```

Observation: To find the current directory in your terminal or command prompt, you can use the "pwd" command in Unix-based systems or "cd" command without any arguments in Windows. It will display the full path of the directory you are currently in.
Thought:

c:\LLMs\MultipleToolKit\toolkit\Lib\site-packages\langchain_google_genai\chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I now know the final answer
Final Answer: You can find your current directory in a terminal by using the "pwd" command in Unix-based systems or "cd" command without any arguments in Windows.

> Finished chain.


'You can find your current directory in a terminal by using the "pwd" command in Unix-based systems or "cd" command without any arguments in Windows.'

In [6]:
loader=WebBaseLoader('https://en.wikipedia.org/wiki/2024_ICC_Men%27s_T20_World_Cup')
data=loader.load()

In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
document=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(data)
vectordb=FAISS.from_documents(document,embeddings)

In [12]:
retriver=vectordb.as_retriever()

In [13]:
retriver

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002252AA7FC50>)

In [14]:
from langchain.tools.retriever import create_retriever_tool

In [104]:
retriver_tool=create_retriever_tool(retriver,"worldcup","Search for Information about T20 worldCup For any Uqestion about WorldCup You must use this Tool")

In [105]:
retriver_tool

Tool(name='worldcup', description='Search for Information about T20 worldCup For any Uqestion about WorldCup You must use this Tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000225196204A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002252AA7FC50>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000022519620680>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002252AA7FC50>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [165]:
tool=[retriver_tool,tools]


tool

[Tool(name='worldcup', description='Search for Information about T20 worldCup For any Uqestion about WorldCup You must use this Tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000225196204A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002252AA7FC50>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000022519620680>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002252AA7FC50>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 [Tool(name='Intermediate Answer', description='useful fo

In [166]:
from langchain import hub
prompt=hub.pull('hwchase17/openai-tools-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [168]:
###Agentss

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm=llm,tools=tool,prompt=prompt)
agent

ValueError: Unsupported function

[Tool(name='Intermediate Answer', description='useful for when you need to ask with search', func=<bound method GoogleSerperAPIWrapper.run of GoogleSerperAPIWrapper(k=10, gl='us', hl='en', type='search', tbs=None, serper_api_key='dc9509e61ea4ac8ee482ef666100c013c0723f8e', aiosession=None, result_key_for_type={'news': 'news', 'places': 'places', 'images': 'images', 'search': 'organic'})>)]

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.

In [118]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor.from_agent_and_tools(name='Agent',agent=agent,tools=tools,verbose=True)

In [162]:
response = agent_executor.invoke({
    "input": "How many teams are playing in 2024 T20 worldcup"
})



> Entering new Agent chain...


KeyError: 'name'